In [1]:
import os
import pandas as pd
from Bio import SeqIO
import yaml
import numpy as np

# load in config with GPSC paths
with open("config/SP_prokka_roary.yaml", 'r') as file:
    config=yaml.safe_load(file)

# extract GPSCs
gpscs = config['samples'] 

# set params
amplicon_stats = list()
xlen = 2200

total_genome_coverages={}
amplicon_positions={}

with open('output.txt', 'w') as f:

    for gpsc, fasta_file in gpscs.items():
        records = list(SeqIO.parse(fasta_file, "fasta"))

        # calculate length of the genome for each GPSC
        genome_length=sum(len(record.seq) for record in records)

        # initialize amplicon genome coverage for each GPSC separately 
        total_genome_coverage=0

        # initialize set of covered positions across each GPSC 
        covered_positions = set()

        # intitialize list of amplicon positions 
        amplicon_positions[gpsc] = []

        print(f"Processing {gpsc}...", file=f)  # Print the name of the GPSC being processed

        # Load the samtools depth file
        depth_file = os.path.join("depth", f"{gpsc}.depth")
        df = pd.read_csv(depth_file, sep="\t", names=["Ref", "Pos", "Depth"])

        # Get the positions where the depth is 1 (primer binding sites)
        primer_binding_sites = df[df["Depth"] == 1]["Pos"].tolist()

        for p1loc in primer_binding_sites:
            # Find the next primer binding site within xlen bases
            p2loc = next((pos for pos in primer_binding_sites if p1loc < pos <= p1loc + xlen), None)

            if p2loc is not None:
                # Calculate the amplicon stats
                amplicon_stats.append((gpsc, gpsc, gpsc, gpsc, gpsc, gpsc, 0, 0, 0, p1loc, p2loc))
                covered_positions.update(range(p1loc, p2loc+1))
                # store amplicon positions 
                amplicon_positions[gpsc].append((p1loc, p2loc))
                print(f"Detected amplicon from {p1loc} to {p2loc}.", file=f)  # Print the start and end positions of the detected amplicon

                # find total length of the amplicon
                total_genome_coverage += p2loc-p1loc # calculate the length of the amplicon and add to total genome coverage

        # calculate predicted % coverage 
        coverage_percentage = (len(covered_positions) / genome_length) * 100

        # update dictionary 
        total_genome_coverages[gpsc] = coverage_percentage

        print(f"Total genome coverage for {gpsc}: {total_genome_coverages[gpsc]}%", file=f)  # Print the total genome coverage for the GPSC

colnames=["pid1","pid2","set1","set2",
          "pseq1","pseq2",
          "max_hdist","hdist1","hdist2",
          "p1loc","p2loc"]
coltypes=["<U30", "<U30", "<U30", "<U30",
          "<U30", "<U30",
          float, float, float,
          int, int]

dt = {'names':colnames, 'formats':coltypes}

amplicon_statsnp = np.array(amplicon_stats,
                     dtype=dt)

np.save("amplicon_statstab.npy", amplicon_statsnp)

In [2]:
for gpsc, coverage in total_genome_coverages.items():
    print(f"{gpsc}: {coverage}%")

GPSC12: 99.28831877584545%
GPSC25: 83.70355584570514%
GPSC31: 83.420166858458%
GPSC34: 86.58638733761124%
GPSC3: 82.58450750499864%
GPSC4: 82.57514313707841%
JYGP01: 37.48834324179536%
GPSC17: 85.52519860790558%
GPSC27: 81.42039075508528%
GPSC32: 86.56638367738053%
GPSC37: 82.37399570281472%
GPSC41: 86.57502063559636%
GPSC5: 82.9434782199731%
NC_017592: 99.28831877584545%


In [3]:
from Bio import SeqIO

# Load the config file
with open("config/SP_prokka_roary.yaml", 'r') as file:
    config=yaml.safe_load(file)

# Extract the GPSCs
gpscs = config['samples']

# Initialize a dictionary to store the genome lengths
genome_lengths = {}

# Calculate and store the length of each sequence
for gpsc, fasta_file in gpscs.items():
    records = list(SeqIO.parse(fasta_file, "fasta"))
    genome_length = sum(len(record.seq) for record in records)
    genome_lengths[gpsc] = genome_length  # Store the genome length in the dictionary

#Print the genome lengths
for gpsc, length in genome_lengths.items():
    print(f"{gpsc}: {length} bp")

GPSC12: 2036867 bp
GPSC25: 2129170 bp
GPSC31: 2085600 bp
GPSC34: 2052251 bp
GPSC3: 2130580 bp
GPSC4: 2121917 bp
JYGP01: 1915198 bp
GPSC17: 2107041 bp
GPSC27: 2160842 bp
GPSC32: 2053788 bp
GPSC37: 2133955 bp
GPSC41: 2001396 bp
GPSC5: 2126260 bp
NC_017592: 2036867 bp


In [ ]:
import matplotlib.pyplot as plt

# Determine the number of rows and columns for the subplots
n = len(amplicon_positions)
ncols = 1
nrows = n // ncols + (n % ncols > 0)

# Create a figure and a grid of subplots
fig, axs = plt.subplots(nrows, ncols, figsize=(10, 5*nrows))

# Flatten the array of axes, in case we have less subplots than planned
axs = axs.flatten()

# Plot the data for each fasta sequence in a separate subplot
for i, (fasta_name, positions) in enumerate(amplicon_positions.items()):
    for start, end in positions:
        axs[i].fill_between([start, end], [1, 1], color='purple')
    axs[i].set_title("Coverage for fasta sequence {}".format(fasta_name))
    axs[i].set_xlabel("Position")
    axs[i].set_xlim(0, genome_lengths[fasta_name])  # Set x limit to the length of the genome
    axs[i].set_ylim(0, 1)

# Remove empty subplots
for i in range(n, nrows*ncols):
    fig.delaxes(axs[i])

# Adjust the layout
plt.tight_layout()

# Save the figure as a PNG file
plt.savefig("fasta_coverage.png")

In [6]:
import pickle

with open('amplicon_positions.pkl', 'wb') as f:
    pickle.dump(amplicon_positions, f)

with open('genome_coverage_pc.pkl', 'wb') as f:
    pickle.dump(total_genome_coverages, f)

In [4]:
# get the branch length of the outgroup since it isn't plotting right when i root it

from Bio import Phylo

# Read the tree from the Newick file
tree = Phylo.read('GPSC_SP_reps.newick', 'newick')

# Get the clade corresponding to the outgroup
outgroup = tree.common_ancestor('JYGP01')

# Get the original branch length of the outgroup
original_branch_length = outgroup.branch_length

print(original_branch_length)

0.065875589


In [ ]:
import matplotlib
matplotlib.use('TkAgg') 

from Bio import Phylo
import matplotlib.pyplot as plt

# Read the tree from the Newick file
tree = Phylo.read('GPSC_SP_reps.newick', 'newick')

# Reroot the tree at the outgroup
tree.root_with_outgroup('JYGP01')

# Get the clade corresponding to the outgroup
outgroup = tree.common_ancestor('JYGP01')

# Manually set the branch length of the outgroup
outgroup.branch_length = 0.063920664

# Draw the tree
Phylo.draw(tree, axes=ax)

# Show the plot
plt.show()

In [22]:
# Plot the predicted amplicon coverage with the percent coverage to the right 

def get_options():
    import argparse

    # Define the command line arguments as variables
    tree = 'GPSC_SP_reps.newick'
    spreadsheet = 'gene_presence_absence.csv'
    depth_dir = '/vast/palmer/scratch/turner/flg9/snakemake_workflows/pangenome_alignment/data/results/samtools_depth'
    labels = False
    format = 'png'
    skipped_columns = 14

    # create the top-level parser
    description = "Create plots from roary outputs"
    parser = argparse.ArgumentParser(description = description,
                                     prog = 'roary_plots.py')

    parser.add_argument('tree', action='store',
                        help='Newick Tree file', default=tree)
    parser.add_argument('spreadsheet', action='store',
                        help='Roary gene presence/absence spreadsheet', default=spreadsheet)
    parser.add_argument('--labels', action='store_true',
                        default=labels,
                        help='Add node labels to the tree (up to 10 chars)')
    parser.add_argument('--format',
                        choices=('png',
                                 'tiff',
                                 'pdf',
                                 'svg'),
                        default=format,
                        help='Output format [Default: png]')
    parser.add_argument('-N', '--skipped-columns', action='store',
                        type=int,
                        default=skipped_columns,
                        help='First N columns of Roary\'s output to exclude [Default: 14]')

    return argparse.Namespace(tree=tree, spreadsheet=spreadsheet, depth_dir=depth_dir, labels=labels, format=format, skipped_columns=skipped_columns)

if __name__ == "__main__":
    options = get_options()

    import matplotlib
    from matplotlib.patches import Rectangle
    from matplotlib.colors import LinearSegmentedColormap
    matplotlib.use('Agg')

    import matplotlib.pyplot as plt
    import seaborn as sns

    sns.set_style('white')

    import os
    import pandas as pd
    import numpy as np
    from Bio import Phylo
    import pickle 

    # Read the tree from the Newick file
    t = Phylo.read(options.tree, 'newick')

    # Reroot the tree at the outgroup
    t.root_with_outgroup('JYGP01')

    # Get the clade corresponding to the outgroup
    outgroup = t.common_ancestor('JYGP01')

    # Manually set the branch length of the outgroup
    outgroup.branch_length = 0.065875589  # replace this with the original branch length

    # check that re-rooting and manual branching occurred 
    print(t)

    # Max distance to create better plots
    mdist = max([t.distance(t.root, x) for x in t.get_terminals()])

    # Load roary
    roary = pd.read_csv(options.spreadsheet, low_memory=False)
    # Set index (group name)
    roary.set_index('Gene', inplace=True)
    # Drop the other info columns
    roary.drop(list(roary.columns[:options.skipped_columns-1]), axis=1, inplace=True)

    # Transform it in a presence/absence matrix (1/0)
    roary.replace('.{2,100}', 1, regex=True, inplace=True)
    roary.replace(np.nan, 0, regex=True, inplace=True)

    # Sort the matrix by the sum of strains presence
    idx = roary.sum(axis=1).sort_values(ascending=False).index
    roary_sorted = roary.loc[idx]

    # Sort the matrix according to tip labels in the tree
    roary_sorted = roary_sorted[[x.name for x in t.get_terminals()]]

    # Calculate the number of terminals
    num_terminals = len(t.get_terminals())

    # Adjust the number of columns in the grid to accommodate the number of terminals
    num_columns = max(60, 3 * num_terminals)
    
    # Plot presence/absence matrix against the tree
    # Create a custom colormap
    cmap = LinearSegmentedColormap.from_list("my_colormap", ["white", "#93a3b1"])

    with sns.axes_style('whitegrid'):
        fig = plt.figure(figsize=(35, 20))  # Adjusted figure size

        ax1 = plt.subplot2grid((1, num_columns), (0, num_columns//3), colspan=num_columns//3)  # Matrix takes up 1/3 of the figure
        a = ax1.matshow(roary_sorted.T, cmap=cmap,
                   vmin=0, vmax=1,
                   aspect='auto',
                   interpolation='none'
                    )
        ax1.set_yticks([])
        ax1.set_xticks([])
        ax1.axis('off')
        ax1.set_title('Roary matrix\n(%d gene clusters)'%roary.shape[0], fontsize = 12)

        # Calculate the number of rows each subplot should span
        num_rows = len(t.get_terminals())
        rows_per_subplot = max(1, num_rows // len(t.get_terminals()))

        # Create subplots for coverage depth plots
        axs_depth = [plt.subplot2grid((num_rows, num_columns), (i * rows_per_subplot, 2*num_columns//3), rowspan=rows_per_subplot, colspan=num_columns//3) for i in range(num_terminals)]  # Depth plots take up 1/3 of the figure

        ax = plt.subplot2grid((1, num_columns), (0, 0), colspan=num_columns//3)  # Tree takes up 1/3 of the figure
        ax.grid(False)

        fig.subplots_adjust(wspace=0, hspace=0)


        if options.labels:
            fsize = 12 - 0.1*roary.shape[1] + 2
            if fsize < 7:
                fsize = 7
            with plt.rc_context({'font.size': fsize}):
                Phylo.draw(t, axes=ax, 
                           show_confidence=False,
                           label_func=lambda x: str(x)[:10],
                           xticks=([],), yticks=([],),
                           ylabel=('',), xlabel=('',),
                           xlim=(-mdist*0.1,mdist+mdist*0.45-mdist*roary.shape[1]*0.001),
                           axis=('off',),
                           title=('Tree\n(%d strains)'%roary.shape[1],), 
                           do_show=False,
                          )
        else:
            Phylo.draw(t, axes=ax, 
                       show_confidence=False,
                       label_func=lambda x: None,
                       xticks=([],), yticks=([],),
                       ylabel=('',), xlabel=('',),
                       xlim=(-mdist*0.1,mdist+mdist*0.1),
                       axis=('off',),
                       title=('Tree\n(%d strains)'%roary.shape[1],), 
                       do_show=False,
                      )


    # Remove grid lines from each subplot 
for ax in axs_depth:
    ax.grid(False)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])

# Remove the space between the subplots
plt.subplots_adjust(wspace=0, hspace=0)

# Plot the coverage depth for each taxa
for i, (ax, terminal) in enumerate(zip(axs_depth, t.get_terminals())):

    # Get the name of the taxa
    taxa_name = terminal.name

    # Load the amplicon_positions dictionary
    with open('amplicon_positions.pkl', 'rb') as f:
        amplicon_positions = pickle.load(f)

    # Check if the taxa has any amplicons
    if taxa_name in amplicon_positions:
        # Iterate over the amplicons
        for p1loc, p2loc in amplicon_positions[taxa_name]:
            # Create the coverage depth plot
            ax.fill_between([p1loc, p2loc], [1, 1], color='#7c898b', alpha=1.0)

    # Get the maximum genome length across all taxa
    max_genome_length = max(genome_lengths.values())

    # Set the x-axis limits to match the range of genome lengths
    ax.set_xlim(0, max_genome_length)

    # Get the coverage percentage for the taxa, default to 0 if not found
    coverage_percentage = total_genome_coverages.get(taxa_name, 0)

    # Get the position of the original axes
    pos = ax.get_position()

    # Create a new axes for the bar plot just to the right of the original axes
    ax_bar = plt.axes([pos.x1 + 0.01, pos.y0, 0.05, pos.height])

    # Plot the horizontal bar on the new axes
    ax_bar.barh(0, coverage_percentage, height=2.0, color='#2d232e')

    # Hide the y-axis
    ax_bar.yaxis.set_visible(False)

    # Set the x-axis limits to match the range of coverage percentages
    ax_bar.set_xlim(0, 100)

    # Set the y-axis limits to match the original axes
    ax_bar.set_ylim(ax.get_ylim())

    # Set x-label
    ax_bar.set_xlabel('Coverage (%)')

    # Remove axes spines 
    for s in ['top', 'right', 'left']:
        ax_bar.spines[s].set_visible(False)

    # Hide x-axis labels and ticks for all but the last ax_bar
    if i < len(axs_depth) - 1:
        ax_bar.xaxis.set_visible(False)
        ax_bar.spines['bottom'].set_visible(False)

    # Remove the title
    ax.set_title('')

    # Set the y-axis limits
    ax.set_ylim(0, 1)  # Set y limit to 1 as the coverage is either 0 or 1

    # Remove the x and y axis labels
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])

    # Remove grid 
    ax.grid(False)

    # Remove seaborn grid
    sns.despine(ax=ax, left=True, bottom=True)

# Get the position of the last ax
pos = ax.get_position()

# Create a new axes for the genome coordinate line just below the last ax
ax_genome = plt.axes([pos.x0, pos.y0, pos.width, 0.0002])

# Hide the y-axis
ax_genome.yaxis.set_visible(False)

# Set the x-axis limits to match the range of genome lengths
ax_genome.set_xlim(0, max_genome_length)

# Set x-axis label
ax_genome.set_xlabel('Genome position (mbp)')

# Remove axes spines 
for s in ['top', 'right', 'left']:
    ax_genome.spines[s].set_visible(False)
            
# Save the plot
plt.savefig('pangenome_matrix_SP_reps_x.%s'%options.format, dpi=300)

Tree(rooted=True, weight=1.0)
    Clade()
        Clade(branch_length=0.065875589, confidence=1.0)
            Clade(branch_length=0.001565992, confidence=1.0)
                Clade(branch_length=0.001067135, confidence=1.0)
                    Clade(branch_length=0.000500928)
                        Clade(branch_length=0.001405424, confidence=1.0)
                            Clade(branch_length=0.004816411, name='GPSC17')
                            Clade(branch_length=0.004338649, name='GPSC37')
                        Clade(branch_length=0.000397671, confidence=0.922)
                            Clade(branch_length=0.001072732, confidence=1.0)
                                Clade(branch_length=0.004405656)
                                    Clade(branch_length=0.0, name='GPSC12')
                                    Clade(branch_length=0.0, name='NC_017592')
                                Clade(branch_length=0.004314957, name='GPSC3')
                            Clade(branch_lengt

/var/folders/v8/xq20p7kj2kd8tzvx4xtdhv65d3pw82/T/ipykernel_5746/3980985191.py:85: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  roary.replace('.{2,100}', 1, regex=True, inplace=True)
